# MOVIE RECOMMENDER ENGINE

## Library

In [1]:
%matplotlib inline
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, HTML
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD


import warnings; warnings.simplefilter('ignore')

## Preprocess DataFrame

In [2]:
md = pd.read_csv('./the-movies-dataset/movies_metadata.csv')

In [3]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
adult                    45466 non-null object
belongs_to_collection    4494 non-null object
budget                   45466 non-null object
genres                   45466 non-null object
homepage                 7782 non-null object
id                       45466 non-null object
imdb_id                  45449 non-null object
original_language        45455 non-null object
original_title           45466 non-null object
overview                 44512 non-null object
popularity               45461 non-null object
poster_path              45080 non-null object
production_companies     45463 non-null object
production_countries     45463 non-null object
release_date             45379 non-null object
revenue                  45460 non-null float64
runtime                  45203 non-null float64
spoken_languages         45460 non-null object
status                   45379 non-null objec

In [4]:
md.head().transpose()[:20]

,0,1,2,3,4
adult,False,False,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,"{'id': 96871, 'name': 'Father of the Bride Col..."
budget,30000000,65000000,0,16000000,0
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 35, 'name': 'Comedy'}]"
homepage,http://toystory.disney.com/toy-story,NaN,NaN,NaN,NaN
id,862,8844,15602,31357,11862
imdb_id,tt0114709,tt0113497,tt0113228,tt0114885,tt0113041
original_language,en,en,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...


### HTML imgage for views

In [5]:
base_poster_url = 'http://image.tmdb.org/t/p/w185/'
md['poster_path'] = "<img src='" + base_poster_url + md['poster_path'] + "' style='height:100px;'>"

In [6]:
md['release_date'][:20]

0     1995-10-30
1     1995-12-15
2     1995-12-22
3     1995-12-22
4     1995-02-10
5     1995-12-15
6     1995-12-15
7     1995-12-22
8     1995-12-22
9     1995-11-16
10    1995-11-17
11    1995-12-22
12    1995-12-22
13    1995-12-22
14    1995-12-22
15    1995-11-22
16    1995-12-13
17    1995-12-09
18    1995-11-10
19    1995-11-21
Name: release_date, dtype: object

### Split date into year values

In [7]:
# when split release_date into 3 element array just take the first element which mean the year 
md['year'] = pd.to_datetime(md['release_date'], errors = 'coerce').apply(lambda x: str(x).split('-')[0] if x!= np.nan else np.nan)

In [8]:
pd.DataFrame(data = md['genres']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 1 columns):
genres    45466 non-null object
dtypes: object(1)
memory usage: 355.3+ KB


In [9]:
md.loc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [10]:
md.loc[md['genres'].notnull() == False]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year


In [11]:
md['genres'] = md['genres'].apply(ast.literal_eval)

In [12]:
md['genres'] = md['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

### Get List Genres

In [13]:
md['genres'][:20]

0            [Animation, Comedy, Family]
1           [Adventure, Fantasy, Family]
2                      [Romance, Comedy]
3               [Comedy, Drama, Romance]
4                               [Comedy]
5       [Action, Crime, Drama, Thriller]
6                      [Comedy, Romance]
7     [Action, Adventure, Drama, Family]
8          [Action, Adventure, Thriller]
9          [Adventure, Action, Thriller]
10              [Comedy, Drama, Romance]
11                      [Comedy, Horror]
12        [Family, Animation, Adventure]
13                      [History, Drama]
14                   [Action, Adventure]
15                        [Drama, Crime]
16                      [Drama, Romance]
17                       [Crime, Comedy]
18            [Crime, Comedy, Adventure]
19               [Action, Comedy, Crime]
Name: genres, dtype: object

In [14]:
s = md.apply(lambda x: pd.Series(x['genres']), axis = 1)

In [15]:
# Series in Pandas - series is one-dimensional labled array of holding any type.
# if data is an nArray : data = np.array(['a','b','c','d']) => s = pd.Series(data)
# => output : 
# 0   a
# 1   b
# 2   c
# 3   d
s[:20]

,0,1,2,3,4,5,6,7
0,Animation,Comedy,Family,NaN,NaN,NaN,NaN,NaN
1,Adventure,Fantasy,Family,NaN,NaN,NaN,NaN,NaN
2,Romance,Comedy,NaN,NaN,NaN,NaN,NaN,NaN
3,Comedy,Drama,Romance,NaN,NaN,NaN,NaN,NaN
4,Comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Action,Crime,Drama,Thriller,NaN,NaN,NaN,NaN
6,Comedy,Romance,NaN,NaN,NaN,NaN,NaN,NaN
7,Action,Adventure,Drama,Family,NaN,NaN,NaN,NaN
8,Action,Adventure,Thriller,NaN,NaN,NaN,NaN,NaN
9,Adventure,Action,Thriller,NaN,NaN,NaN,NaN,NaN


In [16]:
s = pd.DataFrame(data = s)

In [17]:
s = s.stack().reset_index(level=1, drop = True)

### List genres

In [18]:
s[:20]

0    Animation
0       Comedy
0       Family
1    Adventure
1      Fantasy
1       Family
2      Romance
2       Comedy
3       Comedy
3        Drama
3      Romance
4       Comedy
5       Action
5        Crime
5        Drama
5     Thriller
6       Comedy
6      Romance
7       Action
7    Adventure
dtype: object

In [19]:
s.name = 'genre'

In [20]:
gen_md = md.drop('genres', axis = 1).join(s)

In [21]:
gen_md[:20]

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Family
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Adventure
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Fantasy
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Family
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,Romance
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,Comedy
3,False,NaN,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,Comedy
3,False,NaN,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,Drama


In [22]:
gen_md['genre'].value_counts().shape[0]

32

In [23]:
pop_gen = pd.DataFrame(gen_md['genre'].value_counts()).reset_index()

In [24]:
pop_gen.column = ['genre','movies']

In [25]:
pop_gen['index']

0                                     Drama
1                                    Comedy
2                                  Thriller
3                                   Romance
4                                    Action
5                                    Horror
6                                     Crime
7                               Documentary
8                                 Adventure
9                           Science Fiction
10                                   Family
11                                  Mystery
12                                  Fantasy
13                                Animation
14                                  Foreign
15                                    Music
16                                  History
17                                      War
18                                  Western
19                                 TV Movie
20                                  Aniplex
21                Vision View Entertainment
22                              

## Demographic Filtering - Weighted Rating

In [26]:
# IMDB Top Movies Chart Fomula : 
# WR = (v/(v+m).R) + (m/(v+m).C)
# v : số  lượng người bình chọn cho 1 bộ phim 
# m : số lượng bình chọn cần thiếu để được xét vào list - Một phim phải 
# có lượng vote > 95% các phim khác trong list
# R : rating  

In [27]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')

In [28]:
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')

In [29]:
# mean() function can be used to calculate 
# mean/average of a given list of numbers

C = vote_averages.mean()

In [30]:
# rating trung binh tren toan tap dataset
C

5.244896612406511

In [31]:
# Tính phân vị - tìm giá trị m sao cho 95% giá trị trong list bé hơn m
# 5% còn lại lớn hơn nó 
m = vote_counts.quantile(0.95)

In [32]:
m

434.0

In [33]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) 
            & (md['vote_average'].notnull())]

In [34]:
qualified = qualified[['poster_path','title','year','vote_count','vote_average','genres']]

In [35]:
qualified['vote_count'] = qualified['vote_count'].astype('int')

In [36]:
qualified['vote_average'] = qualified['vote_average'].astype('int')

In [37]:
qualified.shape

(2274, 6)

In [38]:
# sumary : một phim cần phải đạt trên 434 votes để  được xét 
#        : điểm rating trung bình là 5.2 
#        : có 2274 phim đạt yêu cầu 

In [39]:
def weighted_rating(x,m,C):
    v = x['vote_count']
    R = x['vote_average']
    
    return (v/(m+v) * R) + (m/(m+v) * C)

In [40]:
qualified['wr'] = qualified.apply(lambda x: weighted_rating(x,m,C), axis=1)

In [41]:
# ascending : tăng dần 

qualified = qualified.sort_values('wr', ascending = False)

In [42]:

HTML(qualified.head(15).to_html(escape = False))

,poster_path,title,year,vote_count,vote_average,genres,wr
15480,,Inception,2010,14075,8,"[Action, Thriller, Science Fiction, Mystery, Adventure]",7.917588
12481,,The Dark Knight,2008,12269,8,"[Drama, Action, Crime, Thriller]",7.905871
22879,,Interstellar,2014,11187,8,"[Adventure, Drama, Science Fiction]",7.897107
2843,,Fight Club,1999,9678,8,[Drama],7.881753
4863,,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,"[Adventure, Fantasy, Action]",7.871787
292,,Pulp Fiction,1994,8670,8,"[Thriller, Crime]",7.868660
314,,The Shawshank Redemption,1994,8358,8,"[Drama, Crime]",7.864000
7000,,The Lord of the Rings: The Return of the King,2003,8226,8,"[Adventure, Fantasy, Action]",7.861927
351,,Forrest Gump,1994,8147,8,"[Comedy, Drama, Romance]",7.860656
5814,,The Lord of the Rings: The Two Towers,2002,7641,8,"[Adventure, Fantasy, Action]",7.851924


In [43]:
gen_md[:20]

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Family
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Adventure
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Fantasy
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Family
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,Romance
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,Comedy
3,False,NaN,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,Comedy
3,False,NaN,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,Drama


In [44]:
gen_md.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93548 entries, 0 to 45465
Data columns (total 25 columns):
adult                    93548 non-null object
belongs_to_collection    11162 non-null object
budget                   93548 non-null object
homepage                 16556 non-null object
id                       93548 non-null object
imdb_id                  93513 non-null object
original_language        93531 non-null object
original_title           93548 non-null object
overview                 92218 non-null object
popularity               93531 non-null object
poster_path              93083 non-null object
production_companies     93539 non-null object
production_countries     93539 non-null object
release_date             93432 non-null object
revenue                  93527 non-null float64
runtime                  93194 non-null float64
spoken_languages         93527 non-null object
status                   93402 non-null object
tagline                  46805 non-null obj

### Filter By Genre

In [45]:
def build_chart(genre, percentile):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())]
    qualified = qualified[['poster_path','title','year','vote_count','vote_average','genre']]
    
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: weighted_rating(x,m,C), axis=1)
    
    qualified = qualified.sort_values('wr', ascending= False)
    
    return qualified



In [46]:
list_genre =  pop_gen['index']
list_genre

0                                     Drama
1                                    Comedy
2                                  Thriller
3                                   Romance
4                                    Action
5                                    Horror
6                                     Crime
7                               Documentary
8                                 Adventure
9                           Science Fiction
10                                   Family
11                                  Mystery
12                                  Fantasy
13                                Animation
14                                  Foreign
15                                    Music
16                                  History
17                                      War
18                                  Western
19                                 TV Movie
20                                  Aniplex
21                Vision View Entertainment
22                              

In [47]:
HTML(build_chart(list_genre.loc[0],0.9).head(15).to_html(escape = False))

,poster_path,title,year,vote_count,vote_average,genre,wr
10309,,Dilwale Dulhania Le Jayenge,1995,661,9,Drama,8.338265
12481,,The Dark Knight,2008,12269,8,Drama,7.969014
22879,,Interstellar,2014,11187,8,Drama,7.966058
2843,,Fight Club,1999,9678,8,Drama,7.960849
314,,The Shawshank Redemption,1994,8358,8,Drama,7.954778
351,,Forrest Gump,1994,8147,8,Drama,7.953629
834,,The Godfather,1972,6024,8,Drama,7.937696
24860,,The Imitation Game,2014,5895,8,Drama,7.936368
359,,The Lion King,1994,5520,8,Drama,7.932163
18465,,The Intouchables,2011,5410,8,Drama,7.930822


## Movie Description Based Recommender

In [48]:
links = pd.read_csv('./the-movies-dataset/links_small.csv')
links[:20]

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
5,6,113277,949.0
6,7,114319,11860.0
7,8,112302,45325.0
8,9,114576,9091.0
9,10,113189,710.0


In [49]:
links = links[links['tmdbId'].notnull()]['tmdbId'].astype('int')
links

0          862
1         8844
2        15602
3        31357
4        11862
         ...  
9120    402672
9121    315011
9122    391698
9123    137608
9124    410803
Name: tmdbId, Length: 9112, dtype: int64

In [50]:
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')


### Filter TMDB contain in Md dataframe

In [51]:
# take all the element that contain id in link dataframe which mean all 
# movie that have tmdbId
smd = md[md['id'].isin(links)]

In [52]:
smd.shape

(9099, 25)

In [53]:
smd.head().transpose()[:20]

,0,1,2,3,4
adult,False,False,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,"{'id': 96871, 'name': 'Father of the Bride Col..."
budget,30000000,65000000,0,16000000,0
genres,"[Animation, Comedy, Family]","[Adventure, Fantasy, Family]","[Romance, Comedy]","[Comedy, Drama, Romance]",[Comedy]
homepage,http://toystory.disney.com/toy-story,NaN,NaN,NaN,NaN
id,862,8844,15602,31357,11862
imdb_id,tt0114709,tt0113497,tt0113228,tt0114885,tt0113041
original_language,en,en,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...


In [54]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')
smd['description'][:20]

0     Led by Woody, Andy's toys live happily in his ...
1     When siblings Judy and Peter discover an encha...
2     A family wedding reignites the ancient feud be...
3     Cheated on, mistreated and stepped on, the wom...
4     Just when George Banks has recovered from his ...
5     Obsessive master thief, Neil McCauley leads a ...
6     An ugly duckling having undergone a remarkable...
7     A mischievous young boy, Tom Sawyer, witnesses...
8     International action superstar Jean Claude Van...
9     James Bond must unmask the mysterious head of ...
10    Widowed U.S. president Andrew Shepherd, one of...
11    When a lawyer shows up at the vampire's doorst...
12    An outcast half-wolf risks his life to prevent...
13    An all-star cast powers this epic look at Amer...
14    Morgan Adams and her slave, William Shaw, are ...
15    The life of the gambling paradise – Las Vegas ...
16    Rich Mr. Dashwood dies, leaving his second wif...
17    It's Ted the Bellhop's first night on the 

## TF-IDF

In [55]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [56]:
tfidf_matrix.shape
# có 9099 vector tương ướng với 9099 overview,
# mỗi vector có 268124 chiều 

(9099, 268124)

In [57]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# Compute the cosine similarity matrix

In [58]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [59]:
cosine_sim[1]

array([0.00680476, 1.        , 0.01531062, ..., 0.00357057, 0.00762326,
       0.        ])

In [60]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [61]:
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
Shin Godzilla                                         9094
The Beatles: Eight Days a Week - The Touring Years    9095
Pokémon: Spell of the Unknown                         9096
Pokémon 4Ever: Celebi - Voice of the Forest           9097
Force Majeure                                         9098
Length: 9099, dtype: int64

In [62]:
def get_recommendations(title):
    # Get the index of the movie that matches the title
    idx = indices[title]
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 10 most similar movies
    #(element number 0 is the movie it self)
    sim_scores = sim_scores[1:11]
    
    
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [63]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [64]:
get_recommendations('The Dark Knight').head(10)


7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object